In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
train_machine = pd.DataFrame()
test_machine = pd.DataFrame()
train_audio_path = []
domain_train = []
train_label = []
train_only_machine = []
test_audio_path = []
domain_test = []
test_label = []
test_only_machine = []
additional_machines = ['3DPrinter', 'AirCompressor', 'BrushlessMotor', 'HairDryer', 'HoveringDrone', 'RoboticArm', 'Scanner', 'ToothBrush', 'ToyCircuit']
for machine in os.listdir('../data/'):
    if machine=='eval_data':
        pass 
    else:
        attributes = pd.read_csv(os.path.join('../data', machine, 'attributes_00.csv'))
        for i, row in attributes.iterrows():
            if 'train' in row['file_name']:
                ########################################
                ## fix additional train set file name ##
                if machine in additional_machines:
                    audio_path = os.path.join(f'data/{machine}/train', row['file_name'])
                    if audio_path[-3:] != 'wav':
                        audio_path += '.wav'
                    train_audio_path.append(audio_path)
                ########################################
                else:
                    train_audio_path.append(os.path.join('data', row['file_name'])) # audio path
                att = '_'.join(row['file_name'].split('.wav')[0].split('_')[6:]) # attribute information
                domain = row['file_name'].split('_')[2] # domain information
                domain_train.append(domain)
                train_label.append('_'.join([machine, domain, att])) # combined label
                train_only_machine.append(machine)
            elif 'test' in row['file_name']:
                ##################################
                ## fix slider no attribute path ##
                if row['file_name'].split('.')[-2][-1] == '_':
                    file_name = row['file_name'].split('.')[0][:-1] + '.wav'
                else:
                    file_name = row['file_name']
                ##################################
                test_audio_path.append(os.path.join('data', file_name)) # audio path
                att = '_'.join(row['file_name'].split('.wav')[0].split('_')[6:]) # attribute information
                if att[-1] == '_':
                    att = att[:-1]
                domain = row['file_name'].split('_')[2] # domain information
                domain_test.append(domain)
                test_label.append('_'.join([machine, domain, att])) # combined label
                test_only_machine.append(machine)

train_machine['audio_path'] = train_audio_path
train_machine['domain'] = domain_train
train_machine['machine_label'] = train_only_machine
train_machine['label'] = train_label
test_machine['audio_path'] = test_audio_path
test_machine['domain'] = domain_test
test_machine['machine_label'] = test_only_machine
test_machine['label'] = test_label

In [4]:
train_machine

,audio_path,domain,machine_label,label
0,data/gearbox/train/section_00_source_train_nor...,source,gearbox,gearbox_source_noAttribute
1,data/gearbox/train/section_00_source_train_nor...,source,gearbox,gearbox_source_noAttribute
2,data/gearbox/train/section_00_source_train_nor...,source,gearbox,gearbox_source_noAttribute
3,data/gearbox/train/section_00_source_train_nor...,source,gearbox,gearbox_source_noAttribute
4,data/gearbox/train/section_00_source_train_nor...,source,gearbox,gearbox_source_noAttribute
...,...,...,...,...
15995,data/AirCompressor/train/section_00_target_tra...,target,AirCompressor,AirCompressor_target_noAttribute
15996,data/AirCompressor/train/section_00_target_tra...,target,AirCompressor,AirCompressor_target_noAttribute
15997,data/AirCompressor/train/section_00_target_tra...,target,AirCompressor,AirCompressor_target_noAttribute
15998,data/AirCompressor/train/section_00_target_tra...,target,AirCompressor,AirCompressor_target_noAttribute


In [3]:
train_machine.to_csv('../processed_data/train.csv', index=None)

In [16]:
train_att = {'audio_path': [], 'domain': [], 'label': []}
att_audio_path = []
att_label = []
for i, row in train_machine.iterrows():
    if 'noAttribute' in row['label']:
        pass
    else:
        train_att['audio_path'].append(row['audio_path'])
        train_att['domain'].append(row['domain'])
        train_att['label'].append(row['label'])
train_att = pd.DataFrame.from_dict(train_att)

In [ ]:
test_att = {'audio_path': [], 'domain': [], 'label': []}
att_audio_path = []
att_label = []
for i, row in test_machine.iterrows():
    if 'noAttribute' in row['label']:
        pass
    else:
        test_att['audio_path'].append(row['audio_path'])
        test_att['domain'].append(row['domain'])
        test_att['label'].append(row['label'])
test_att = pd.DataFrame.from_dict(test_att)

In [ ]:
train_machine.to_csv('../processed_data/train_machine.csv', index=None)
test_machine.to_csv('../processed_data/test_machine.csv', index=None)
train_att.to_csv('../processed_data/train_att.csv', index=None)
test_att.to_csv('../processed_data/test_att.csv', index=None)

---

In [4]:
train = pd.DataFrame()
test = pd.DataFrame()
train_audio_path = []
train_machine = []
train_domain = []
test_audio_path = []
test_machine = []
test_label = []
test_domain = []
for machine in os.listdir('../data'):
    if machine=='eval_data':
        pass
    else:
        folder_path = os.path.join('../data', machine)
        for flag in os.listdir(folder_path):
            if flag == 'train':
                file_path = os.path.join('../data', machine, 'train')
                for file in os.listdir(file_path):
                    train_audio_path.append(os.path.join(file_path[3:], file))
                    train_machine.append(machine)
                    if 'source' in file:
                        train_domain.append('source')
                    elif 'target' in file:
                        train_domain.append('target')
            elif flag == 'test':
                file_path = os.path.join('../data', machine, 'test')
                for file in os.listdir(file_path):
                    test_audio_path.append(os.path.join(file_path[3:], file))
                    test_machine.append(machine)
                    if 'normal' in file:
                        test_label.append(0)
                    elif 'anomaly' in file:
                        test_label.append(1)
                    if 'source' in file:
                        test_domain.append('source')
                    elif 'target' in file:
                        test_domain.append('target')
            else:
                pass

train['audio_path'] = train_audio_path
train['machine'] = train_machine
train['domain'] = train_domain
test['audio_path'] = test_audio_path
test['machine'] = test_machine
test['label'] = test_label
test['domain'] = test_domain

In [7]:
eval = pd.DataFrame()
eval_audio_path = []
machine_label = []
domain = []
for machine in os.listdir('../data/eval_data'):
    folder_path = os.path.join('../data/eval_data', machine, 'test')
    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path[3:], file)
        eval_audio_path.append(file_path)
        machine_label.append(machine)
            
eval['audio_path'] = eval_audio_path
eval['machine'] = machine_label

In [13]:
train.to_csv('../processed_data/evaluation_train.csv', index=None)
test.to_csv('../processed_data/evaluation_test.csv', index=None)
eval.to_csv('../processed_data/evaluation_eval.csv', index=None)

In [42]:
from sklearn.preprocessing import LabelEncoder
train_data = pd.read_csv('../processed_data/train_att.csv')
train_data = train_data[train_data['domain']=='source'].reset_index(drop=True)

# Label encoding
le = LabelEncoder()
le.fit(train_data['label'])
train_data['label'] = le.transform(train_data['label'])
output_size = len(le.classes_)

In [43]:
output_size

87